

# ML Study Jam Exercise 4 - HUMAN EMOTION DETECTION

## Install necessary Libraries

In [ ]:
%pip install numpy opencv-python tensorflow pandas

## Setup Kaggle Library

In [ ]:
# install kaggle libary
%pip install -q kaggle

In [ ]:
# upload Kaggle API Credentials
from google.colab import files

files.upload()

In [ ]:
# add kaggle API credentials to root
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# create a new folder to save dataset
! mkdir ./kaggleDataset

# set Directory as current directory
%cd ./kaggleDataset

In [ ]:
# list kaggle datasets
! kaggle datasets list

## Install Dataset

In [ ]:
import os

# create new dir for dataset
! mkdir ./fer2013

# set Directory as current directory
%cd ./fer2013

In [ ]:
# download the dataset
!kaggle datasets download -d msambare/fer2013

In [ ]:
# Complete path to storage location of the .zip file of data
zip_path = '/content/kaggleDataset/fer2013/fer2013.zip'
# Check current directory (be sure you're in the directory where Colab operates: '/content')
os.getcwd()
# Copy the .zip file into the present directory
!cp '{zip_path}' .
# Unzip quietly
!unzip -q 'fer2013.zip'
# View the unzipped contents in the virtual machine
os.listdir()

## Data Analysis

In [ ]:
# Set up file paths
TRAIN_PATH='/content/kaggleDataset/fer2013/train'
TEST_PATH='/content/kaggleDataset/fer2013/test'

In [ ]:
category_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

category_info = []

for category in category_names:
    path = os.path.join(TRAIN_PATH, category)
    class_num = category_names.index(category)
    count = 1
    for img in os.listdir(path):
      count +=1
    category_info.append({
        'label': category,
        'Count': count
    })
    print(category + " : " + str(count))

In [ ]:
# plot histogram for labels

import matplotlib.pyplot as plt

labels = [item['label'] for item in category_info]
counts = [item['Count'] for item in category_info]

plt.bar(labels, counts)
plt.xlabel('Categories')
plt.ylabel('Count')
plt.title('Histogram of Category Counts')
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
plt.show()

In [ ]:
from tensorflow.keras.utils import load_img

plt.figure(figsize=(14,22))
i = 1
for expression in os.listdir(TRAIN_PATH):
    img = load_img((TRAIN_PATH + '/' + expression +'/'+ os.listdir(TRAIN_PATH + '/' + expression)[6]))
    plt.subplot(1,7,i)
    plt.imshow(img)
    plt.title(expression)
    plt.axis('off')
    i += 1
plt.show()

In [ ]:
import cv2
import matplotlib.pyplot as plt

img_array = cv2.imread('train/happy/Training_39155692.jpg')
img_array.shape

In [ ]:
plt.imshow(img_array)

In [ ]:
new_array = cv2.resize(img_array, (224, 224))
plt.imshow(cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB))
plt.show()

## Data Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
"""
Data Augmentation
--------------------------
rotation_range = rotates the image with the amount of degrees we provide
width_shift_range = shifts the image randomly to the right or left along the width of the image
height_shift range = shifts image randomly to up or below along the height of the image
horizontal_flip = flips the image horizontally
rescale = to scale down the pizel values in our image between 0 and 1
zoom_range = applies random zoom to our object
validation_split = reserves some images to be used for validation purpose
"""

train_datagen = ImageDataGenerator(width_shift_range = 0.1,
                                  height_shift_range = 0.1,
                                  horizontal_flip = True,
                                  rescale = 1./255,
                                  validation_split = 0.2
                                  )
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

In [ ]:
"""
Applying data augmentation to the images as we read
them from their respective directories
"""


img_size = 48 #original size of the image

#%% MODEL HYPERPARAMETER BATCH_SIZE [32 - 256] (Multiples of 2) %%
batch_size = 128


train_generator = train_datagen.flow_from_directory(directory = TRAIN_PATH,
                                                    target_size = (img_size,img_size),
                                                    batch_size = batch_size,
                                                    color_mode = "grayscale", # rgb for transfer learning
                                                    class_mode = "categorical",
                                                    subset = "training"
                                                   )
validation_generator = validation_datagen.flow_from_directory( directory = TEST_PATH,
                                                              target_size = (img_size,img_size),
                                                              batch_size = batch_size,
                                                              color_mode = "grayscale", # rgb for transfer learning
                                                              class_mode = "categorical",
                                                              subset = "validation"
                                                             )

## Model Construction

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras import regularizers
import tensorflow as tf

### Build A Model

In [ ]:
"""
RESNET Modeling

def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x

def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x

shape = (48, 48, 1)
classes = 7

# Step 1 (Setup Input Layer)
x_input = tf.keras.layers.Input(shape)
x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)

# Step 2 (Initial Conv layer along with maxPool)
x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)

# Define size of sub-blocks and initial filter size
block_layers = [3, 4, 6, 3]
filter_size = 64

# Step 3 Add the Resnet Blocks
for i in range(4):
    if i == 0:
        # For sub-block 1 Residual/Convolutional block not needed
        for j in range(block_layers[i]):
            x = identity_block(x, filter_size)
    else:
        # One Residual/Convolutional Block followed by Identity blocks
        # The filter size will go on increasing by a factor of 2
        filter_size = filter_size*2
        x = convolutional_block(x, filter_size)
        for j in range(block_layers[i] - 1):
            x = identity_block(x, filter_size)

# Step 4 End Dense Network
x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)

model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")

"""

In [ ]:
"""
Modeling


model = Sequential()
model.add(Conv2D(filters = 64,kernel_size = (3,3),padding = 'same',activation = 'relu',input_shape=(img_size,img_size,1)))
model.add(MaxPooling2D(pool_size = 2,strides = 2))
model.add(BatchNormalization())

model.add(Conv2D(filters = 128,kernel_size = (3,3),padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2,strides = 2))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128,kernel_size = (3,3),padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2,strides = 2))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters = 256,kernel_size = (3,3),padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2,strides = 2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(units = 128,activation = 'relu',kernel_initializer='he_normal'))
model.add(Dropout(0.25))
model.add(Dense(units = 64,activation = 'relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(units = 32,activation = 'relu',kernel_initializer='he_normal'))
model.add(Dense(7,activation = 'softmax'))

"""

In [ ]:
model = Sequential()

# Convolutional layer 1  -- > input_shape=(48, 48, 1) for grayscale input_shape=(48, 48, 3) for rgb
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48, 1)))  # Input shape: (48, 48, 1), Output shape: (48, 48, 32)

# Convolutional layer 2
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))  # Output shape: (48, 48, 64)

# Batch Normalization
model.add(BatchNormalization())  # Output shape: (48, 48, 64)

# Max Pooling
model.add(MaxPooling2D(2, 2))  # Input shape: (48, 48, 64), Output shape: (24, 24, 64)

# Dropout
model.add(Dropout(0.25))  # Input shape: (24, 24, 64), Output shape: (24, 24, 64)

# Convolutional layer 3 with L2 regularization
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)))  # Input shape: (24, 24, 64), Output shape: (24, 24, 128)

# Convolutional layer 4 with L2 regularization
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))  # Input shape: (24, 24, 128), Output shape: (22, 22, 256)

# Batch Normalization
model.add(BatchNormalization())  # Output shape: (22, 22, 256)

# Max Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))  # Input shape: (22, 22, 256), Output shape: (11, 11, 256)

# Dropout
model.add(Dropout(0.25))  # Input shape: (11, 11, 256), Output shape: (11, 11, 256)

# Flatten
model.add(Flatten())  # Input shape: (11, 11, 256), Output shape: 30976

# Fully Connected layer 1
model.add(Dense(1024, activation='relu'))  # Input shape: 30976, Output shape: 1024

# Dropout
model.add(Dropout(0.5))  # Input shape: 1024, Output shape: 1024

# Fully Connected layer 2
model.add(Dense(7, activation='softmax'))  # Input shape: 1024, Output shape: 7


### Transfer Learning

In [ ]:
model_MobileNet = tf.keras.applications.MobileNetV2()
base_input = model_MobileNet.layers[0].input
base_output = model_MobileNet.layers[-2].output

In [ ]:
final_output = layers.Dense(128)(base_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation='softmax')(final_output)

In [ ]:
model = keras.Model(inputs = base_input, outputs = final_output)

## Model Summary

In [ ]:
#%% MODEL HYPERPARAMETER LEARNING RATE [0.1 - 0.001 - 0.0001] %%
#%% MODEL HYPERPARAMETER WEIGHT DECAY [0.0001 - 0.001] (Multiples of 2) %%

model.compile(
    optimizer = Adam(learning_rate=0.0001, decay=1e-6),
    loss='categorical_crossentropy',
    metrics=['accuracy']
  )

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

## Model Training

In [ ]:
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, ReduceLROnPlateau
import datetime

chk_path = 'model.h5'
log_dir = "checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             moniter='val_loss')

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=6,
                              verbose=1,
                              min_delta=0.0001)


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, reduce_lr, csv_logger]

In [ ]:
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size

hist = model.fit(x=train_generator,
                 validation_data=validation_generator,
                 epochs=60, # 30 for TL
                 callbacks=callbacks,
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

## Model Evaluation

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(1,2,2)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'], loc='upper left')

plt.subplot(1,2,1)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
train_loss, train_accu = model.evaluate(train_generator)
test_loss, test_accu = model.evaluate(validation_generator)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_accu*100, test_accu*100))

In [ ]:
import numpy as np

y_pred = model.predict(train_generator)
y_pred = np.argmax(y_pred, axis=1)
class_labels = validation_generator.class_indices
class_labels = {v:k for k,v in class_labels.items()}

from sklearn.metrics import classification_report, confusion_matrix
cm_train = confusion_matrix(train_generator.classes, y_pred)
print('Confusion Matrix')
print(cm_train)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(train_generator.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_train, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)

## Model Testing

In [ ]:
from keras.utils import load_img
img = load_img(TEST_PATH + "img path here ", target_size = (48,48), color_mode = "grayscale")
img = np.array(img)
plt.imshow(img)
print(img.shape)

In [ ]:
category_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [ ]:
from keras.utils import img_to_array
test_image = img_to_array(img)
test_image = np.expand_dims(test_image, axis = 0)
prediction = model.predict(test_image)
prediction[0]

In [ ]:
res = np.argmax(prediction[0])
print('predicted Label for that image is: {}'.format(category_names[res]))

## Model Deployment

In [ ]:
model.save('model_seq_optimal.h5')

In [ ]:
model.save_weights('model_weights.h5')